In [279]:
import numpy as np
import torch
import torchvision
from torchvision import transforms
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [280]:
from torchsampler import ImbalancedDatasetSampler
# In the folder "test_images" you can/should combine "google_faces" and "yale_faces" and rename it to "1"  (faces) and the other one "google_images02_36x36" to "0" (non-faces)
train_dir = './train_images'
test_dir = './test_images'

transform = transforms.Compose(
    [transforms.Grayscale(), 
     transforms.ToTensor(), 
     transforms.Normalize(mean=(0,),std=(1,))])

train_data = torchvision.datasets.ImageFolder(train_dir, transform=transform)
test_data = torchvision.datasets.ImageFolder(test_dir, transform=transform)

valid_size = 0.2
batch_size = 32

num_train = len(train_data)
indices_train = list(range(num_train))
np.random.shuffle(indices_train)
split_tv = int(np.floor(valid_size * num_train))
train_new_idx, valid_idx = indices_train[split_tv:],indices_train[:split_tv]

# train_sampler = SubsetRandomSampler(train_new_idx)
# valid_sampler = SubsetRandomSampler(valid_idx)

train_sampler = ImbalancedDatasetSampler(train_data, train_new_idx)
valid_sampler = ImbalancedDatasetSampler(train_data, valid_idx)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, sampler=train_sampler, num_workers=1)
valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, sampler=valid_sampler, num_workers=1)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=True, num_workers=1)
classes = ('noface','face')

In [281]:
# for epoch in range(1, n_epochs+1):
# for data, target in train_loader:*

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=200, kernel_size=3, padding=1)  # notice the padding
        self.conv2 = nn.Conv2d(in_channels=200, out_channels=500, kernel_size=3, padding=1) # again...        
        self.pool = nn.MaxPool2d(2,2)
        self.fc1 = nn.Linear(500*9*9, 300) # it is 64....
        self.fc2 = nn.Linear(300, 100)
        self.fc3 = nn.Linear(100, 2)
        self.dropout = torch.nn.Dropout(p=0.5)
        self.relu = torch.nn.ReLU()

    # Defining the forward pass    
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = F.max_pool2d(x, (2,2))
        x = self.conv2(x) 
        x = self.relu(x)
        x = F.max_pool2d(x, 2) 
        #print(x.shape)
        x = x.reshape(x.size(0), -1) 
        
        x = self.fc1(x) 
        x = self.dropout(x)
        x = self.fc2(x) 
        x = self.dropout(x)
        prediction = self.fc3(x) 
        return prediction      

device = "cpu"
if (torch.cuda.is_available()):
    device = "cuda"

net = Net()
print(net)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Net(
  (conv1): Conv2d(1, 200, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(200, 500, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=40500, out_features=300, bias=True)
  (fc2): Linear(in_features=300, out_features=100, bias=True)
  (fc3): Linear(in_features=100, out_features=2, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (relu): ReLU()
)


In [274]:
# # for epoch in range(1, n_epochs+1):
# # for data, target in train_loader:*

# import torch
# import torch.nn as nn
# import torch.nn.functional as F


# class Net(nn.Module):
#     def __init__(self):
#         super(Net, self).__init__()
#         # 1 input image channel, 6 output channels, 3x3 square convolution
#         # kernel
#         self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=3,stride=1, padding=1)
#         self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=3, stride=1, padding=1)

#         self.pool = nn.MaxPool2d(kernel_size=2)
#         self.drop = nn.Dropout2d(p=0.2) #drop layer deletes 20% of features to help prevent overfitting
        
#         self.fc = nn.Linear(16 * 7 * 7, 2)
        
# #         # an affine operation: y = Wx + b
# #         self.fc = nn.Linear(16* 7 * 7, 120)  # 6*6 from image dimension
# #         self.fc2 = nn.Linear(120, 84)
# #         self.fc3 = nn.Linear(in_features = 84, out_features = 2)
         
#     def forward(self, x):
#         x = F.relu(self.pool(self.conv1(x)))
#         x = F.relu(self.pool(self.conv2(x)))
#         x = F.dropout(self.drop(x), training = self.training)
#         x = x.view(-1, self.num_flat_features(x))
#         x = self.fc(x)
        
#         return torch.log_softmax(x, dim=1)
# #         # Max pooling over a (2, 2) window
# #         x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
# #         # If the size is a square you can only specify a single number
# #         x = F.max_pool2d(F.relu(self.conv2(x)), (2, 2))
# #         x = F.max_pool2d(F.relu(self.conv3(x)), 2)

# #         x = x.view(-1, self.num_flat_features(x))
# #         x = F.relu(self.fc1(x))
# #         x = F.relu(self.fc2(x))
# #         x = self.fc3(x)
# #         return x

#     def outputSize(in_size, kernel_size, stride, padding):
#         output = int((in_size - kernel_size + 2*(padding)) / stride) + 1
#         return(output)

#     def num_flat_features(self, x):
#         size = x.size()[1:]  # all dimensions except the batch dimension
#         num_features = 1
#         for s in size:
#             num_features *= s
#         return num_features

# device = "cpu"
# if (torch.cuda.is_available()):
#     device = "cuda"

# net = Net()
# print(net)
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [282]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.01)

In [285]:
net.to(device)

n_epoch = int(num_train/batch_size)
print(n_epoch)
for epoch in range(3):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

net = Net()
net.to(device)

PATH = './cifar_net.pth'
torch.save(net.state_dict(), PATH)

2897
[1,  2000] loss: 0.693
[2,  2000] loss: 0.693
[3,  2000] loss: 0.693
Finished Training


In [ ]:
net = Net()

net.load_state_dict(torch.load(PATH))

correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total))